In [1]:
import pandas as pd
import numpy as np
from env import conn
import matplotlib as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
%matplotlib inline
cs = conn.cursor()

In [2]:
cs.execute("SELECT * FROM sandbox.cm_sandbox.lc_links_joins_forecast")

In [3]:
query_result = cs.fetchall()

In [4]:
column_names = [col[0] for col in cs.description]

In [5]:
cs.close()

True

In [6]:
df = pd.DataFrame(query_result, columns=column_names)

In [7]:
df.head()

,DATE,CHANNEL,LOYALTY_PLAN_COMPANY,CUMULATIVE_JOINS
0,2023-11-04,LLOYDS,Stonegate Group,27
1,2023-11-12,LLOYDS,Stonegate Group,30
2,2023-11-02,LLOYDS,Stonegate Group,27
3,2023-10-26,LLOYDS,Stonegate Group,12
4,2023-10-22,LLOYDS,Stonegate Group,6


In [9]:
df.dtypes

DATE                    datetime64[ns]
CHANNEL                         object
LOYALTY_PLAN_COMPANY            object
CUMULATIVE_JOINS                 int64
dtype: object

In [10]:
df.columns

Index(['DATE', 'CHANNEL', 'LOYALTY_PLAN_COMPANY', 'CUMULATIVE_JOINS'], dtype='object')

In [11]:
df.set_index('DATE', inplace=True)

In [21]:
y = df['CUMULATIVE_JOINS']

In [22]:
train_size = int(len(y) * 0.7)
train, test = y[0:train_size], y[train_size:len(y)]

In [23]:
model = ARIMA(train, order=(5,1,0)) # EDIT THESE NUMBERS TO FIT MODEL
model_fit = model.fit()

/Users/chrismitchell/codespace/jupyter/venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/chrismitchell/codespace/jupyter/venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/chrismitchell/codespace/jupyter/venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [24]:
forecast = model_fit.forecast(steps=len(test))

/Users/chrismitchell/codespace/jupyter/venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/chrismitchell/codespace/jupyter/venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [26]:
forecast.head()

24    17.434018
25    25.601695
26    16.313089
27    14.459217
28    22.925085
Name: predicted_mean, dtype: float64